# H20 automl: Porto Seguro (Classification) # 1

Based on: https://www.kaggle.com/code/unfinishedgod/automl-h2o-practice-with-safe-driver-prediction

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "AUC"  # specify your evaluation metric
target = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from h2o.automl import H2OAutoML

import h2o
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
# Import data

train_set = pd.read_csv("porto_train.csv")
test_set = pd.read_csv("porto_test.csv")

In [4]:
train_set.shape

(476170, 60)

In [5]:
test_set.shape

(119042, 59)

In [6]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_372"; OpenJDK Runtime Environment (build 1.8.0_372-b07); OpenJDK 64-Bit Server VM (build 25.372-b07, mixed mode)
  Starting server from /home/stever7/.local/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_cvabice
  JVM stdout: /tmp/tmp_cvabice/h2o_stever7_started_from_python.out
  JVM stderr: /tmp/tmp_cvabice/h2o_stever7_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,1 year and 5 months !!!
H2O_cluster_name:,H2O_from_python_stever7_7vs0a0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.63 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


## Tidy

In [7]:
train_set = train_set.replace(-1, np.nan)
test_set = test_set.replace(-1, np.nan)

## Transform

In [8]:
# Convert categorical variables to "category" data type (after converting to h20 dataframe - easier)
cat_vars = [col for col in train_set.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
train_set = h2o.H2OFrame(train_set)
test_set = h2o.H2OFrame(test_set)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
for col in cat_vars:
    test_set[col] = test_set[col].asfactor()
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_set[col] = train_set[col].asfactor()

In [11]:
predictors = train_set.columns
predictors.remove(target)  # remove "target"
predictors.remove("id")  # remove "id" (should not be a predictor)
predictors.remove("fold")  # remove "id" (should not be a predictor)

## Visualize

(skipped)

## Model

In [12]:
%%time

automl_h2o_models = H2OAutoML(max_runtime_secs=time_limit, seed=1, sort_metric=metric)

CPU times: user 3.21 ms, sys: 696 µs, total: 3.91 ms
Wall time: 27.7 ms


In [13]:
%%time

automl_h2o_models.train(x=predictors, y=target, training_frame=train_set)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

16:36:26.347: GBM_lr_annealing_selection_AutoML_1_20230717_153637 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

CPU times: user 24.4 s, sys: 2.07 s, total: 26.4 s
Wall time: 1h 4s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_1_20230717_153637

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0331709942627501
RMSE: 0.18212905935833001
LogLoss: 0.1407390223313069
Null degrees of freedom: 9852
Residual degrees of freedom: 9810
Null deviance: 3043.629546252695
Residual deviance: 2773.403174060734
AIC: 2859.403174060734
AUC: 0.7539015953481437
AUCPR: 0.16255114953974145
Gini: 0.5078031906962874

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.07350317146240423: 


,,0,1,Error,Rate
0,0,9108.0,392.0,0.0413,(392.0/9500.0)
1,1,257.0,96.0,0.728,(257.0/353.0)
2,Total,9365.0,488.0,0.0659,(649.0/9853.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.073503,0.228300,108.0
1,max f2,0.047775,0.287737,193.0
2,max f0point5,0.092257,0.233415,73.0
3,max accuracy,0.193281,0.964782,7.0
4,max precision,0.252451,1.000000,0.0
5,max recall,0.014462,1.000000,380.0
6,max specificity,0.252451,1.000000,0.0
7,max absolute_mcc,0.073503,0.197614,108.0
8,max min_per_class_accuracy,0.038772,0.679887,238.0
9,max mean_per_class_accuracy,0.039597,0.685589,234.0



Gains/Lift Table: Avg response rate:  3.58 %, avg score:  3.61 %


...

,,0,1,Error,Rate
0,0,416668.0,42143.0,0.0919,(42143.0/458811.0)
1,1,13613.0,3746.0,0.7842,(13613.0/17359.0)
2,Total,430281.0,45889.0,0.1171,(55756.0/476170.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.060971,0.118454,184.0
1,max f2,0.037600,0.199752,270.0
2,max f0point5,0.078089,0.099188,142.0
3,max accuracy,0.412928,0.963551,7.0
4,max precision,0.515304,0.750000,1.0
5,max recall,0.010079,1.000000,398.0
6,max specificity,0.539589,0.999998,0.0
7,max absolute_mcc,0.045738,0.081612,235.0
8,max min_per_class_accuracy,0.035298,0.600090,281.0
9,max mean_per_class_accuracy,0.037152,0.600917,272.0



Gains/Lift Table: Avg response rate:  3.65 %, avg score:  3.65 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010003,0.111494,3.507309,3.507309,0.127861,0.140611,0.127861,0.140611,0.035083,0.035083,250.730907,250.730907,0.026029
1,2,0.020001,0.094572,2.592696,3.050099,0.094518,0.101917,0.111193,0.121268,0.025923,0.061006,159.269637,205.009875,0.042556
2,3,0.030002,0.085118,2.286854,2.795684,0.083368,0.089495,0.101918,0.110677,0.022870,0.083876,128.685402,179.568384,0.055912
3,4,0.040000,0.078691,2.223957,2.652775,0.081075,0.081724,0.096708,0.103440,0.022236,0.106112,122.395733,165.277473,0.068613
4,5,0.050001,0.073941,2.171647,2.556545,0.079168,0.076235,0.093200,0.097999,0.021718,0.127830,117.164727,155.654519,0.080774
5,6,0.100000,0.060045,1.873419,2.214989,0.068296,0.066093,0.080748,0.082046,0.093669,0.221499,87.341915,121.498934,0.126096
6,7,0.150001,0.052489,1.531162,1.987044,0.055819,0.055982,0.072439,0.073358,0.076560,0.298059,53.116204,98.704372,0.153659
7,8,0.200000,0.047357,1.377988,1.834783,0.050235,0.049778,0.066888,0.067463,0.068898,0.366957,37.798850,83.478311,0.173273
8,9,0.300000,0.040393,1.204562,1.624710,0.043913,0.043624,0.059230,0.059517,0.120456,0.487413,20.456247,62.470956,0.194504
9,10,0.400000,0.035518,1.060545,1.483668,0.038663,0.037834,0.054088,0.054096,0.106054,0.593467,6.054496,48.366841,0.200787


In [14]:
%%time

lb = automl_h2o_models.leaderboard

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 11.7 µs


In [15]:
%%time

# automl_h2o_models.leaderboard
lb.head(rows=lb.nrows)

CPU times: user 3.56 ms, sys: 0 ns, total: 3.56 ms
Wall time: 5.77 ms


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_1_20230717_153637,0.641497,0.151821,0.0671654,0.438028,0.186381,0.0347379
StackedEnsemble_AllModels_3_AutoML_1_20230717_153637,0.641381,0.151835,0.0672132,0.437836,0.186383,0.0347386
StackedEnsemble_BestOfFamily_4_AutoML_1_20230717_153637,0.639227,0.151982,0.0660236,0.429874,0.186431,0.0347566
StackedEnsemble_AllModels_2_AutoML_1_20230717_153637,0.637625,0.152089,0.0653894,0.429973,0.186458,0.0347665
XGBoost_grid_1_AutoML_1_20230717_153637_model_16,0.637397,0.152084,0.0658065,0.436697,0.186446,0.0347621
StackedEnsemble_BestOfFamily_6_AutoML_1_20230717_153637,0.637329,0.152582,0.0647298,0.434916,0.186569,0.0348082
StackedEnsemble_BestOfFamily_3_AutoML_1_20230717_153637,0.637133,0.152114,0.0652895,0.431051,0.186464,0.0347687
XGBoost_grid_1_AutoML_1_20230717_153637_model_23,0.636553,0.152138,0.0655165,0.431297,0.186462,0.0347679
StackedEnsemble_AllModels_1_AutoML_1_20230717_153637,0.635232,0.152293,0.0635048,0.431319,0.186528,0.0347928
GBM_grid_1_AutoML_1_20230717_153637_model_14,0.635137,0.152344,0.0639354,0.43322,0.186528,0.0347928


In [16]:
automl_h2o_models.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_1_20230717_153637

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0331709942627501
RMSE: 0.18212905935833001
LogLoss: 0.1407390223313069
Null degrees of freedom: 9852
Residual degrees of freedom: 9810
Null deviance: 3043.629546252695
Residual deviance: 2773.403174060734
AIC: 2859.403174060734
AUC: 0.7539015953481437
AUCPR: 0.16255114953974145
Gini: 0.5078031906962874

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.07350317146240423: 


,,0,1,Error,Rate
0,0,9108.0,392.0,0.0413,(392.0/9500.0)
1,1,257.0,96.0,0.728,(257.0/353.0)
2,Total,9365.0,488.0,0.0659,(649.0/9853.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.073503,0.228300,108.0
1,max f2,0.047775,0.287737,193.0
2,max f0point5,0.092257,0.233415,73.0
3,max accuracy,0.193281,0.964782,7.0
4,max precision,0.252451,1.000000,0.0
5,max recall,0.014462,1.000000,380.0
6,max specificity,0.252451,1.000000,0.0
7,max absolute_mcc,0.073503,0.197614,108.0
8,max min_per_class_accuracy,0.038772,0.679887,238.0
9,max mean_per_class_accuracy,0.039597,0.685589,234.0



Gains/Lift Table: Avg response rate:  3.58 %, avg score:  3.61 %


...

,,0,1,Error,Rate
0,0,416668.0,42143.0,0.0919,(42143.0/458811.0)
1,1,13613.0,3746.0,0.7842,(13613.0/17359.0)
2,Total,430281.0,45889.0,0.1171,(55756.0/476170.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.060971,0.118454,184.0
1,max f2,0.037600,0.199752,270.0
2,max f0point5,0.078089,0.099188,142.0
3,max accuracy,0.412928,0.963551,7.0
4,max precision,0.515304,0.750000,1.0
5,max recall,0.010079,1.000000,398.0
6,max specificity,0.539589,0.999998,0.0
7,max absolute_mcc,0.045738,0.081612,235.0
8,max min_per_class_accuracy,0.035298,0.600090,281.0
9,max mean_per_class_accuracy,0.037152,0.600917,272.0



Gains/Lift Table: Avg response rate:  3.65 %, avg score:  3.65 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010003,0.111494,3.507309,3.507309,0.127861,0.140611,0.127861,0.140611,0.035083,0.035083,250.730907,250.730907,0.026029
1,2,0.020001,0.094572,2.592696,3.050099,0.094518,0.101917,0.111193,0.121268,0.025923,0.061006,159.269637,205.009875,0.042556
2,3,0.030002,0.085118,2.286854,2.795684,0.083368,0.089495,0.101918,0.110677,0.022870,0.083876,128.685402,179.568384,0.055912
3,4,0.040000,0.078691,2.223957,2.652775,0.081075,0.081724,0.096708,0.103440,0.022236,0.106112,122.395733,165.277473,0.068613
4,5,0.050001,0.073941,2.171647,2.556545,0.079168,0.076235,0.093200,0.097999,0.021718,0.127830,117.164727,155.654519,0.080774
5,6,0.100000,0.060045,1.873419,2.214989,0.068296,0.066093,0.080748,0.082046,0.093669,0.221499,87.341915,121.498934,0.126096
6,7,0.150001,0.052489,1.531162,1.987044,0.055819,0.055982,0.072439,0.073358,0.076560,0.298059,53.116204,98.704372,0.153659
7,8,0.200000,0.047357,1.377988,1.834783,0.050235,0.049778,0.066888,0.067463,0.068898,0.366957,37.798850,83.478311,0.173273
8,9,0.300000,0.040393,1.204562,1.624710,0.043913,0.043624,0.059230,0.059517,0.120456,0.487413,20.456247,62.470956,0.194504
9,10,0.400000,0.035518,1.060545,1.483668,0.038663,0.037834,0.054088,0.054096,0.106054,0.593467,6.054496,48.366841,0.200787


In [18]:
perf = automl_h2o_models.leader.model_performance(test_set)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.03471850697392254
RMSE: 0.1863290288009964
LogLoss: 0.15196055916360815
Null degrees of freedom: 119041
Residual degrees of freedom: 118999
Null deviance: 37231.77510716302
Residual deviance: 36179.377767908474
AIC: 36265.377767908474
AUC: 0.6354875315493407
AUCPR: 0.06647831169274243
Gini: 0.27097506309868136

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.053103478664332064: 


,,0,1,Error,Rate
0,0,98687.0,16020.0,0.1397,(16020.0/114707.0)
1,1,3058.0,1277.0,0.7054,(3058.0/4335.0)
2,Total,101745.0,17297.0,0.1603,(19078.0/119042.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.053103,0.118066,192.0
1,max f2,0.039640,0.198181,246.0
2,max f0point5,0.088623,0.101986,104.0
3,max accuracy,0.356242,0.963618,5.0
4,max precision,0.356242,0.833333,5.0
5,max recall,0.009552,1.000000,398.0
6,max specificity,0.451102,0.999991,0.0
7,max absolute_mcc,0.053103,0.082348,192.0
8,max min_per_class_accuracy,0.034749,0.592257,270.0
9,max mean_per_class_accuracy,0.039640,0.596611,246.0



Gains/Lift Table: Avg response rate:  3.64 %, avg score:  3.62 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010005,0.111404,3.527693,3.527693,0.128463,0.137921,0.128463,0.137921,0.035294,0.035294,252.769299,252.769299,0.026245
1,2,0.020001,0.094275,2.861448,3.194710,0.104202,0.101976,0.116338,0.119956,0.028604,0.063899,186.144786,219.471033,0.045556
2,3,0.030006,0.085241,2.121227,2.836782,0.077246,0.089392,0.103303,0.109765,0.021223,0.085121,112.122716,183.678243,0.057198
3,4,0.040003,0.078990,1.938400,2.612281,0.070588,0.081910,0.095128,0.102804,0.019377,0.104498,93.840016,161.228119,0.066933
4,5,0.050008,0.074177,2.029000,2.495586,0.073887,0.076503,0.090879,0.097542,0.020300,0.124798,102.899989,149.558574,0.077617
5,6,0.100007,0.060088,1.803952,2.149798,0.065692,0.066174,0.078286,0.081859,0.090196,0.214994,80.395188,114.979786,0.119333
6,7,0.150006,0.052355,1.711678,2.003766,0.062332,0.055942,0.072969,0.073221,0.085582,0.300577,71.167812,100.376612,0.156261
7,8,0.200005,0.047184,1.259537,1.817717,0.045867,0.049644,0.066193,0.067327,0.062976,0.363552,25.953673,81.771659,0.169728
8,9,0.300003,0.040253,1.215707,1.617052,0.044271,0.043455,0.058886,0.059370,0.121569,0.485121,21.570670,61.705225,0.192114
9,10,0.400002,0.035268,0.973488,1.456165,0.035450,0.037620,0.053027,0.053932,0.097347,0.582468,-2.651190,45.616459,0.189362


In [ ]:
# Predict on test set
# pred_conversion = automl_h2o_models.predict(test_set)
# pred_conversion = pred_conversion.as_data_frame()

## Communicate

(skipped)